Notebook to be executed locally.

In [1]:
import keras
import cv2

Using TensorFlow backend.


In [17]:
def softMaxAxis1(x):
    return keras.activations.softmax(x, axis=1)

In [8]:
from keras.layers import Input, Dense, Conv2D, BatchNormalization, \
MaxPool2D, UpSampling2D, Concatenate, ReLU, Dropout

In [9]:
inp = Input(shape=(96,96,1))
enc1 = Conv2D(16, (3,3), padding="same")(inp)
enc1 = BatchNormalization()(enc1)
enc1 = Dropout(0.2)(enc1)
enc1 = ReLU()(enc1)
p1 = MaxPool2D()(enc1)
enc2 = Conv2D(32, (3,3), padding="same")(p1)
enc2 = BatchNormalization()(enc2)
enc2 = Dropout(0.2)(enc2)
enc2 = ReLU()(enc2)
p2 = MaxPool2D()(enc2)
enc3 = Conv2D(48, (3,3), padding="same")(p2)
enc3 = BatchNormalization()(enc3)
enc3 = Dropout(0.2)(enc3)
enc3 = ReLU()(enc3)
p3 = MaxPool2D()(enc3)
dec4 = Conv2D(48, (3,3), padding="same")(p3)
dec4 = BatchNormalization()(dec4)
dec4 = Dropout(0.2)(dec4)
dec4 = ReLU()(dec4)
q3 = UpSampling2D()(dec4)
q3 = Concatenate()([q3, p2, enc3])
dec3 = Conv2D(32, (3,3), padding="same")(q3)
dec3 = BatchNormalization()(dec3)
dec3 = Dropout(0.2)(dec3)
dec3 = ReLU()(dec3)
q2 = UpSampling2D()(dec3)
q2 = Concatenate()([q2, p1, enc2])
dec2 = Conv2D(16, (3,3), padding="same", activation="relu")(q2)
dec2 = BatchNormalization()(dec2)
dec2 = Dropout(0.2)(dec2)
dec2 = ReLU()(dec2)
q1 = UpSampling2D()(dec2)
q1 = Concatenate()([q1, inp, enc1])
dec1 = Conv2D(15, (3,3), padding="same", 
              activation=lambda x: keras.activations.softmax(x,axis=-1))(q1)

point_extractor = keras.Model(inp, dec1)
point_extractor.compile(optimizer=keras.optimizers.Adam(), 
        loss = keras.losses.categorical_crossentropy)

In [18]:
from keras.layers import Conv2D, Dropout
model = keras.models.Sequential()
model.add(Conv2D(16, (3,3), padding="same", 
        activation="relu", input_shape=(96,96,1)))
model.add(Dropout(0.2))
model.add(Conv2D(24, (3,3), padding="same", 
        activation="relu"))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3,3), padding="same", 
        activation="relu"))
model.add(Dropout(0.2))
model.add(Conv2D(40, (3,3), padding="same", 
        activation="relu"))
model.add(Dropout(0.2))
model.add(Conv2D(15, (3,3), padding="same",
        activation=softMaxAxis1))

Loading weights of the model trained on google colab.

In [12]:
point_extractor.load_weights("face_weights_9")

In [17]:
def predict(frame):
    frame = frame.reshape(1, 96, 96, 1)
    points = point_extractor.predict(frame)[0]
    point_coords = np.unravel_index\
    (np.argmax(points.reshape(-1,15), axis = 0), (96, 96))
    point_coords = np.array(point_coords).T
    return point_coords

Running trained model on faces detected with Haar cascades.

In [14]:
face_detector = cv2.CascadeClassifier('C:\\Users\\hunte\\Anaconda3\\envs\\env2\\Library\\etc\\haarcascades\\haarcascade_frontalface_default.xml')

In [15]:
import numpy as np

In [19]:
dev = cv2.VideoCapture(0)
try:
    while(True):
        ret, frame = dev.read()
        if (not ret):
            continue
        xSize, ySize, _ = frame.shape
        cv2.imshow("webcam", frame)
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_detector.detectMultiScale(frame_gray)
        if (len(faces)>0):
            (x,y,w,h) = faces[0]
            cv2.rectangle(frame, (x,y), (x+w, y+h), [0,0,255], 10)
            face = frame_gray[y:y+h, x:x+w]/255
            face_resized = cv2.resize(face, (96, 96))
            coords = predict(face_resized)
            for i in range(15):
                face_resized = cv2.circle(face_resized, 
                                          (coords[i][1],coords[i][0]) 
                                           , 3, (255,0,0))
            cv2.imshow("new face", face_resized)
        
            
        cv2.imshow("faces", frame)
        if (cv2.waitKey(1) == ord('q')):
            break
except Exception as e:
    print(e)

dev.release()
cv2.destroyAllWindows() 